In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer,KNNImputer,MissingIndicator
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeClassifier
plt.style.use('ggplot')
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder,KBinsDiscretizer,FunctionTransformer
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('titanic.csv')

In [4]:
df.drop(columns=['Name','SibSp','Parch','Ticket','Cabin'],inplace=True)


In [5]:
df

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked
0,1,0,3,male,22.0,7.2500,S
1,2,1,1,female,38.0,71.2833,C
2,3,1,3,female,26.0,NaN,S
3,4,1,1,female,35.0,NaN,S
4,5,0,3,male,35.0,8.0500,S
...,...,...,...,...,...,...,...
518,519,1,2,female,36.0,26.0000,S
519,520,0,3,male,32.0,7.8958,S
520,521,1,1,female,30.0,93.5000,S
521,522,0,3,male,22.0,7.8958,S


In [6]:
x=df.drop(columns=['Survived'])
y=df['Survived']


In [7]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

### pipeline new methodb

In [8]:
x_train

,PassengerId,Pclass,Sex,Age,Fare,Embarked
220,221,3,male,16.00,8.0500,S
69,70,3,male,26.00,8.6625,S
360,361,3,male,40.00,27.9000,S
305,306,1,male,0.92,151.5500,S
173,174,3,male,21.00,7.9250,S
...,...,...,...,...,...,...
71,72,3,female,16.00,46.9000,S
106,107,3,female,21.00,7.6500,S
270,271,1,male,NaN,31.0000,S
435,436,1,female,14.00,120.0000,S


In [37]:
numerical_feature=['Age','Fare']
numerical_transformer=Pipeline(steps=[
    ('age',SimpleImputer(strategy='median')),
    ('fare',StandardScaler()),
    ('binning', KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile'))
    
    
    
])

categorical_feature=['Sex','Embarked']
categorical_transformer=Pipeline(steps=[
    ('embark',SimpleImputer(strategy='most_frequent')),
    ('sex',OneHotEncoder(handle_unknown='ignore'))
])


iterative_feature =['Age']
iterative=Pipeline(steps=[
    
    ('step',IterativeImputer())
])

In [38]:
process = ColumnTransformer([
   ('numerical', numerical_transformer,numerical_feature),
   ('categorical',categorical_transformer,categorical_feature),
   ('iterative',iterative,iterative_feature)
    
    ],remainder='passthrough')

In [39]:
clf= Pipeline(steps=[
    
    
    ('prc',process),
    ('de',DecisionTreeClassifier())
])

In [40]:
clf.fit(x_train,y_train)

,steps,"[('prc', ...), ('de', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numerical', ...), ('categorical', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [41]:
y_pred= clf.predict(x_test)

In [42]:
accuracy_score(y_test,y_pred)

0.7333333333333333

In [78]:


# Numerical pipeline
numerical_feature = ['Age', 'Fare']
numerical_transformer = Pipeline(steps=[
    ('iterative',IterativeImputer()),
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_feature = ['Sex', 'Embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine using ColumnTransformer
process = ColumnTransformer([
    ('numerical', numerical_transformer, numerical_feature),
    ('categorical', categorical_transformer, categorical_feature)
], remainder='passthrough')

# Final pipeline
clf = Pipeline(steps=[
    ('preprocessor', process),
    ('classifier', DecisionTreeClassifier())
])


In [79]:
clf.fit(x_train,y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numerical', ...), ('categorical', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [49]:
y_pred=clf.predict(x_test)

In [50]:
accuracy_score(y_test,y_pred)

0.7142857142857143

PassengerId     0
Pclass          0
Sex             0
Age            83
Fare            4
Embarked        1
dtype: int64

In [54]:
x

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,1,3,male,22.0,7.2500,S
1,2,1,female,38.0,71.2833,C
2,3,3,female,26.0,NaN,S
3,4,1,female,35.0,NaN,S
4,5,3,male,35.0,8.0500,S
...,...,...,...,...,...,...
518,519,2,female,36.0,26.0000,S
519,520,3,male,32.0,7.8958,S
520,521,1,female,30.0,93.5000,S
521,522,3,male,22.0,7.8958,S


In [21]:
x_train

,PassengerId,Pclass,Sex,Age,Fare,Embarked
220,221,3,male,16.00,8.0500,S
69,70,3,male,26.00,8.6625,S
360,361,3,male,40.00,27.9000,S
305,306,1,male,0.92,151.5500,S
173,174,3,male,21.00,7.9250,S
...,...,...,...,...,...,...
71,72,3,female,16.00,46.9000,S
106,107,3,female,21.00,7.6500,S
270,271,1,male,NaN,31.0000,S
435,436,1,female,14.00,120.0000,S


In [64]:
def p(bins,str):
    
   
    
    
    Clf = ColumnTransformer([
        
    ('iterative',IterativeImputer(),[3]),   
    ('age_impute', SimpleImputer(strategy='mean'), [3]),
    ('age_bin', KBinsDiscretizer(n_bins=bins, strategy=str, encode='ordinal'), [3]),
    
    ('fare_impute', SimpleImputer(strategy='mean'), [4]),
    ('fare_bin', KBinsDiscretizer(n_bins=bins, strategy=str, encode='ordinal'), [4]),
    
    ('pclass_log', FunctionTransformer(np.log1p), [0])
     ], remainder='passthrough')

    x_cross = Clf.fit_transform(x_train)
    np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10))
    
    
    
    fig, (ax1,ax2,ax3,ax4)= plt.subplots(ncols=4,figsize=(15,3.1))

    sns.histplot(x_train['Age'],ax=ax1,kde=True,color='yellow')
    ax1.set_title("before")
    
    sns.histplot(x_cross[:,3],kde=True,color='red',ax=ax2)
    
    sns.histplot(x_train['Fare'],ax=ax3,kde=True,color='yellow')
    ax3.set_title("tip before")
    
    sns.histplot(x_cross[:,4],kde=True,color='red',ax=ax4)
    ax4.set_title("after")
    


In [67]:
clf.fit(x_train,y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numerical', ...), ('categorical', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [72]:
y_pred= clf.predict(x_test)

In [73]:
accuracy_score(y_test,y_pred)

0.6952380952380952

In [69]:
x_train.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            83
Fare            4
Embarked        1
dtype: int64

In [71]:
p(10,'kmeans')

ValueError: Input X contains NaN.
KBinsDiscretizer does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values